### 3-1. 문장의 어조(tone_s)를 구한 후 문서 전체의 어조(tone_i)를 구한다.

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
from ekonlpy.tag import Mecab
from ekonlpy.sentiment import MPCK
import pandas as pd
import numpy as np
mpck = MPCK()

In [2]:
# 단어 사전을 불러온다.
pos_dic = pd.read_csv('../datasets/3_pos_dict.csv')
neg_dic = pd.read_csv('../datasets/3_neg_dict.csv')

In [3]:
# 병합한 데이터를 불러온다.
df_3 = pd.read_csv('../datasets/1_minutes_with_label.csv')
del df_3['Unnamed: 0']
df_3 = df_3.dropna()
df_3

,date,title,content,ngram,label
0,2008-04-10,금융통화위원회 의사록(2008년도 제8차),"일부 위원은 금년 하반기중 국내경제가 소,｢ ｣ 프트패치 로 평가된 년 분기부터 년...","['중/NNG', '경제/NNG', '소/NNG', '트/NNG', '패치/NNG'...",down
1,2008-05-08,금융통화위원회 의사록(2008년도 제10차),"일부 위원은 최근 경기상승세가 둔화되고,｢ ｣ 있는 가운데 일각에서는 경기가 금년 ...","['경기/NNG', '상승/NNG', '둔화/NNG', '되/XSV', '있/VA'...",down
3,2008-06-12,금융통화위원회 의사록(2008년도 제12차),"일부 위원은 금년 월중 제조업 및 서비, 4 5｢ ｣ ～ 스업 생산활동이 비교적 괜...","['월중/NNG', '제조업/NNG', '업/NNG', '생산/NNG', '활동/N...",down
5,2008-07-10,금융통화위원회 의사록(2008년도 제14차),일부 위원은 최근 물가오름세가 빨라지고 있음에도 불구하고 긴축정책을 펴는 데에 ...,"['물가/NNG', '오름/NNG', '빠르/VA', '불구/NNG', '긴축정책/...",up
7,2008-08-07,금융통화위원회 의사록(2008년도 제16차),"일부 위원은 금리정책이 물가에 미치는 영,｢ ｣ 향이나 파급시차에 관해서는 많은 연...","['물가/NNG', '미치/VV', '영/NNG', '파급시차/NNG', '관해서/...",up
...,...,...,...,...,...
241,2020-01-17,제1차_금통위 의사록,일부 위원은 미국 ISM제조업지수의 최근 월 발표내용을 보면 다수의 산업에 서 수...,"['ism/NNG', '제조업/NNG', '지수/NNG', '발표/NNG', '내용...",same
242,2020-02-27,제4차_금통위 의사록,일부 위원은 국내경제가 올해 회복세로 전환될 것이라는 당초 전망의 중요한 전제 중...,"['경제/NNG', '회복/NNG', '전환/NNG', '되/VV', '전망/NNG...",down
246,2020-04-09,제8차_금통위 의사록,일부 위원은 관련부서에서 전망경로의 상하방 위험을 대체로 균형있게 고려 하고 있는...,"['전망/NNG', '경로/NNG', '상하방/NNG', '위험/NNG', '대체로...",down
249,2020-05-28,제12차_금통위 의사록,일부 위원은 코로나19 사태의 전개양상을 둘러싼 불확실성이 매우 높은 상황 에서 ...,"['코로나/NNG', '사태/NNG', '전개/NNG', '양상/NNG', '둘러싸...",down


In [6]:
# 금통위 의사록 중 아무 데이터나 하나 고른다.
contents = df_3._get_value(246, 'content') 
contents = contents.replace('  ','').split('.')

In [7]:
# 톤을 구한다.
def tone(haw, dov):
    # tone 공식 : (No.of Hawkish - No.of Dovish) / (No.of Hawkish + No.of Dovish)
    bunja =  haw - dov
    bunmo =  haw + dov
    try:
        tone = bunja / bunmo
    except:
        tone = 0
    
    return tone 

In [8]:
# 문장 내의 haw와 dov의 개수를 센다.
def cnt_haw_dov(ngram):
    haw, dov = 0, 0
    
    for n in ngram:
        if n in haw_ls:
            haw+=1
        elif n in dov_ls:
            dov+=1
                    
    tone_s = tone(haw, dov)
    
    return np.where(tone_s>0, 'H', 'D')

In [9]:
haw_ls = pos_dic['ngram'].values.tolist()
dov_ls = neg_dic['ngram'].values.tolist()

Haw , Dov = 0, 0

for sents in contents:
    
    tokens = mpck.tokenize(sents)
    ngrams = mpck.ngramize(tokens)

    ngram = tokens+ngrams

    label = cnt_haw_dov(ngram)

    if label == 'H':
        Haw +=1
    else:
        Dov +=1
        
# 문서 전체의 톤을 구한다.
print('Haw : {}, Dov : {}'.format(Haw, Dov))         
tone_i = tone(Haw, Dov)
print('tone_i = ',tone_i)
print('Hawkish' if tone_i > 0 else 'Dovish')

Haw : 89, Dov : 198
tone_i =  -0.3797909407665505
Dovish
